In [19]:
from dataset.model_loader import ModelLoader
from datasets import load_dataset
from bert_routing.train_BERT import ModelTrainer
import pickle
import argparse
import random
import os
from config import DatasetConfig, MODEL_REGISTRY, TrainingConfig
from itertools import product
import torch
import gc

def load_pickle_data(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

if __name__ == "__main__":
    print('Hello World')
    task = 'train'
    if task == 'train':
        # get the arguments
        model_name = TrainingConfig.model_name
        data_size = TrainingConfig.data_size
        batch_size = TrainingConfig.batch_size
        context_window = TrainingConfig.context_window
        num_epochs = TrainingConfig.num_epochs
        strategy = TrainingConfig.strategy

        index = 0
        # Load test data
        test_path = os.path.join(DatasetConfig.DATA_DIR, DatasetConfig.TEST_FILE)
        test_data = load_pickle_data(test_path)
        test_texts = [sample['text'] for sample in test_data]
        test_labels = [[sample['labels'][index]] for sample in test_data]

        # Load and shuffle train data
        train_path = os.path.join(DatasetConfig.DATA_DIR, DatasetConfig.TRAIN_FILE)
        train_data = load_pickle_data(train_path)
        random.shuffle(train_data)
        train_texts = [sample['text'] for sample in train_data]
        train_labels = [[sample['labels'][index]] for sample in train_data]
        
        if data_size != 'None':
            train_texts = train_texts[:int(data_size)]
            train_labels = train_labels[:int(data_size)]
        num_classes = 2

        learning_rates = [1e-5, 1e-6]
        layers_to_freeze_options = [2, 4]

        grid = product(learning_rates, layers_to_freeze_options)
        for lr, layers in grid:
            
            TrainingConfig.learning_rate = lr
            TrainingConfig.layers_to_freeze = layers

            trainer = ModelTrainer(model_name=model_name,
                num_outputs=len(train_labels[0]),
                num_classes=num_classes,
                pooling_strategy=strategy, 
                train_texts=train_texts,
                train_labels=train_labels,
                test_texts=test_texts,
                test_labels=test_labels)

            trainer.train(batch_size=batch_size, 
                context_window=context_window, 
                num_epochs=num_epochs,)


            # Clean up to avoid GPU memory leak
            del trainer
            torch.cuda.empty_cache()
            gc.collect()

Hello World


AttributeError: type object 'TrainingConfig' has no attribute 'model_name'

In [18]:
import os
print('salam')
dir_path = os.getcwd()  # current working directory
print(dir_path)

salam
/data/gpfs/projects/punim2662/LLM_routing/LLM_routing
